In [19]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm as cmap
%matplotlib inline

In [39]:
def ResizeWithAspectRatio(image, width=None, height=None, inter=cv2.INTER_AREA):
        dim = None
        (h, w) = image.shape[:2]

        if width is None and height is None:
            return image
        if width is None:
            r = height / float(h)
            dim = (int(w * r), height)
        else:
            r = width / float(w)
            dim = (width, int(h * r))
        return cv2.resize(image, dim, interpolation=inter)

In [38]:
# Load image
image = cv2.imread('Images/road.jpg')
if image.shape[0] > 700:
    image = ResizeWithAspectRatio(image,height=600)
elif image.shape[1] > 700:
    image = ResizeWithAspectRatio(image,width=600)
image_copied = image.copy()

In [42]:
# Segment for drawing using watershed algorithm
segments = np.zeros(image.shape,dtype=np.uint8)

In [43]:
# Draw marker color on original image
marker = np.zeros(image.shape[:2],dtype=np.int32)

In [44]:
# Use ColorMap
colors = []
for i in range(10):
    # Convert color range to 0-255
    colors.append(tuple(np.array(cmap.tab10(i)[:3])*255))

In [41]:
def mouse_callback(event,x,y,flags,param):
    global marker_updated
    if event == cv2.EVENT_LBUTTONDOWN:
        # Draw circle on mouse left click
        cv2.circle(marker,(x,y),10,(marker_now),-1)
        cv2.circle(image_copied,(x,y),10,colors[marker_now],-1)

        marker_updated = True

In [45]:
marker_updated = False
marker_now = 1

In [48]:
cv2.namedWindow('Image')
cv2.setMouseCallback('Image',mouse_callback)

while True:
    # Display 2 windows
    cv2.imshow('Image',image_copied)
    cv2.imshow('Watershed Algorithm',segments)

    key_pressed = cv2.waitKeyEx(1)
    if key_pressed == 27:
        break
    elif key_pressed == ord('c'):
        image_copied = image.copy()
        marker = np.zeros(image.shape[:2],dtype=np.int32)
        segments = np.zeros(image.shape,dtype=np.uint8)
    elif key_pressed > 0 and chr(key_pressed).isdigit():
        marker_now = int(chr(key_pressed))

    if marker_updated:
        # Apply wathershed algorithm
        marker_copied = marker.copy()
        cv2.watershed(image,marker_copied)
        segments = np.zeros(image.shape,dtype=np.uint8)

        for color in range(10):
            segments[marker_copied == (color)] = colors[color]
cv2.destroyAllWindows()